# Food prediction model: L1

In [ ]:
# import packages 
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers


from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from keras.utils.vis_utils import plot_model


import cv2
import os
import gc
import random
import matplotlib.image as mpimg

#


## Profide a model name

In [ ]:
model_name = 'J2'

## Load labels and paths

In [ ]:
# file location
train_label_path = '../input/train-labels/train_labels.csv'
train_image_path = '../input/food-stuff/train_set/train_set/'
test_image_path= '../input/test-set-2/test_set'
test_results_sample = '../input/sample/sample.csv'

# importing files
df_train_labels = pd.read_csv(train_label_path, sep=',')
df_test_labels = pd.read_csv(test_results_sample, sep=',')

# sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

In [ ]:
df_test_labels.head()

## Image sample

In [ ]:
# not working on kaggle

## Data pre-procession

In [ ]:
# Adding the file paths to the dataframe for train
path_list = []
for i in range(len(df_train_labels)):
    path = '{}'.format(df_train_labels['img_name'][i])
    path_list.append(path)

    
df_train_labels['path'] = path_list
df_train_labels['label'] = df_train_labels['label'].astype(str)

del path_list

# Adding the file paths to the dataframe for test
path_list = []
for i in range(len(df_test_labels)):
    path = '{}'.format(df_test_labels['img_name'][i])
    path_list.append(path)

    
df_test_labels['path'] = path_list
df_test_labels['label'] = df_test_labels['label'].astype(str)


In [ ]:
df_test_labels.head()

## Train and validation set

In [ ]:
image_train, image_validation = train_test_split(df_train_labels,
                                                 test_size=0.2, 
                                                 random_state=12, 
                                                 stratify=df_train_labels['label'])


In [ ]:
image_train.head()

# Loading images training and validation

In [ ]:
# setting image parameters
img_height = 128
img_width = 128
image_size = (img_height,img_width)
batch_size = 120

# defining Generators
datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True,
                             rotation_range=30, zoom_range=0.2)
dataval =ImageDataGenerator()
test = ImageDataGenerator()

# parameters train data
train_images = datagen.flow_from_dataframe(
    image_train,
    directory = train_image_path,
    class_mode='categorical',
    x_col='path',
    y_col='label',
    shuffle=True,
    target_size=(img_height, img_width),
    batch_size=batch_size
)

#  parameters validation data
validation_images = dataval.flow_from_dataframe(
    image_validation,
    directory = train_image_path,
    class_mode='categorical',
    x_col='path',
    y_col='label',
    shuffle=True,
    target_size=(img_height, img_width),
    batch_size=batch_size
)

#  parameters test data
test_generator = test.flow_from_directory(
        test_image_path,
        target_size=image_size,
        batch_size=batch_size,
        shuffle=False
)

In [ ]:
def create_model(base_model):
    base_model.trainable = True
    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output) # kan weg?
    prediction_layer = tf.keras.layers.Dense(num_classes, activation='sigmoid')(global_average_layer)
    model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001), loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=["accuracy"])
    return model

In [ ]:
from sklearn.utils import class_weight 

class_weights = class_weight.compute_class_weight(
           'balanced',
            np.unique(train_images.classes), 
            train_images.classes)

train_class_weights = dict(enumerate(class_weights))
print()

In [ ]:
batch_size = 32
epochs = 20
num_classes = 80

# setting the location for train results
checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 monitor = 'val_accuracy',
                                                 save_best_only=True,
                                                 verbose=1)

def fit_model(model):    
    history = model.fit(train_images,
                        validation_data = validation_images,
                        epochs=epochs,
                        class_weight=train_class_weights,
                        callbacks=[cp_callback])
    return history

IMG_SHAPE = (224, 224, 3)
base_model1 = tf.keras.applications.EfficientNetB2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")
base_model2 = tf.keras.applications.EfficientNetB3(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")
base_model3 = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

model1 = create_model(base_model1)
model2 = create_model(base_model2)
model3 = create_model(base_model3)
                        
                        
history1 = fit_model(model1)
model1.save('models/model1.h5')
history2 = fit_model(model2)
model2.save('models/model2.h5')
history3 = fit_model(model3)
model3.save('models/model3.h5')



In [ ]:
## load best models for magic

In [ ]:
def load_all_models():
    all_models = []
    model_names = ['model1.h5', 'model2.h5', 'model3.h5']
    for model_name in model_names:
        filename = os.path.join('models', model_name)
        model = tf.keras.models.load_model(filename)
        all_models.append(model)
        print('loaded:', filename)
    return all_models

def ensemble_model(models):
    for i, model in enumerate(models):
        for layer in model.layers:
            layer.trainable = False
    ensemble_visible = [model.input for model in models]
    ensemble_outputs = [model.output for model in models]
    merge = tf.keras.layers.concatenate(ensemble_outputs)
    merge = tf.keras.layers.Dense(10, activation='relu')(merge)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(merge)
    model = tf.keras.models.Model(inputs=ensemble_visible, outputs=output)
    
    
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), 
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
                  metrics=["accuracy"])
    return model

models = load_all_models()
for i, model in enumerate(models):
    for layer in model.layers:
        layer.trainable = False
        
        

In [ ]:
X = [X_train for _ in range(len(model.input))]
X_1 = [X_test for _ in range(len(model.input))]

In [ ]:
epochs = 20
history = model.fit(X, y_train,
                    batch_size=batch_size,
                    steps_per_epoch=len(total_train) // batch_size,
                    epochs=epochs,
                    validation_data=(X_1, y_test),
                    validation_steps=len(total_val) // batch_size
                    )



In [ ]:
model.save('models/model.h5')

print('MobileNetV2 acc:', history1.history['val_accuracy'][-1])
print('InceptionV3 acc:', history2.history['val_accuracy'][-1])
print('Xception acc:', history3.history['val_accuracy'][-1])
print('Ensemble acc:', history.history['val_accuracy'][-1])

## Model design

In [ ]:
# Number of output layers/classes
num_classes = 80

# load models



#Setting the input size
input_shape = (img_height,img_width,3)

# from tensorflow.keras.applications import InceptionResNetV2

model = tf.keras.applications.EfficientNetB2(weights='imagenet',
                      include_top=False,
                      input_shape=input_shape,
                      pooling="avg",
                      classes=num_classes)




# Since we load pre-trained model we must specify first layer as non-trainable
# model.layers[0].trainable = True
model.trainable = True

inputs = model.input

x = tf.keras.layers.Dense(128, activation='relu')(model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)

outputs = tf.keras.layers.Dense(80, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

# compiling model
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(learning_rate = 0.0001),
              metrics=['accuracy'])



In [ ]:
# model overview
model.summary()


## Model training

In [ ]:
# setting the location for train results
checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)


In [ ]:
from sklearn.utils import class_weight 

class_weights = class_weight.compute_class_weight(
           'balanced',
            np.unique(train_images.classes), 
            train_images.classes)

train_class_weights = dict(enumerate(class_weights))
print()

In [ ]:
# setting the number of training iterations
epochs=30

# saving substeps
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 monitor = 'val_accuracy',
                                                 save_best_only=True,
                                                 verbose=1)

# initiate training
history = model.fit(
  train_images,
  validation_data = validation_images,
  epochs=epochs,
  class_weight=train_class_weights,
  callbacks=[cp_callback]
)

model.save("EfficientNetB2_30ep")

In [ ]:
# Loads the weights
# model = tf.saved_model.load('./training_1')
# Re-evaluate the model 
# loss, acc = model.evaluate(test_images, test_labels, verbose=2)./training_1/cp.ckpt/saved_model.pb
# print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

# checkpoint_dir ../output/kaggle/working/trainin_1/cp.ckpt/saved_model.pb'

## Accuracy plots

In [ ]:
# subsetting all hyper results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

# plotting Training and Validation Accuracy
plt.figure(figsize=(14, 14))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

# plotting the Training and Validation Loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()
plt.savefig('testim.png')

## Predicting on the testset

In [ ]:
def predict_for_test(model):
    '''
    This functions tests the model on the test set and 
    automaticaly maps the resulting labels to a .csv file.
    
    '''
    # getting labels
    labels = (train_images.class_indices)
    labels = dict((v,k) for k,v in labels.items())

    # predicting on the test set
    preds = model.predict(test_generator)
    preds_cls_idx = preds.argmax(axis=-1)
    predictions = [labels[k] for k in preds_cls_idx]

    #m apping predictions and save as df
    filenames=test_generator.filenames
    filenames = [x.replace('test_set/', '') for x in filenames]
    results=pd.DataFrame({"img_name":filenames,
                          "label":predictions})
    return results 

test_results_pr = predict_for_test(model)
test_results = df_test_labels.merge(test_results_pr, how='left', on='img_name')
test_results = test_results[['path', 'label_y']].rename(columns = {'label_y':'label', 'path':'img_name'})


# writing pridictions to csv
test_results.to_csv("sample_submission_model_J1.csv", index=False)


In [ ]:
# showing the prediction dist
plt.figure(figsize=(20,15))
sns.countplot(x='label', data=test_results).set(title='Count per label')
plt.show()
# plt.savefig('model_1_label_dist.png')


In [ ]:
# printing the predictions
test_results

## Notes

In [ ]:
# Loads the weights
# model.load_weights(checkpoint_path)
# model.save("J2_50_ep.h5")

In [ ]:
# model = Sequential()
# model.add(Conv2D(32, (3, 3), padding='same',
#                  input_shape=(32,32,3)))
# model.add(Activation('relu'))
# model.add(Conv2D(32, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Conv2D(64, (3, 3), padding='same'))
# model.add(Activation('relu'))
# model.add(Conv2D(64, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Flatten())
# model.add(Dense(512))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(10, activation='softmax'))
# model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])